In [72]:
import os
from tqdm.auto import tqdm

In [101]:
config_name = 'small_tune_ve5_linear_stop_after_completes'
path = f"/home/upload/ray_results/{config_name}/"
trials = os.listdir(path)
trials = [os.path.join(path, trial) for trial in trials]
trials = sorted(filter(os.path.isdir, trials))
print(f"Got {len(trials)} trials")

Got 635 trials


In [108]:
# trying to re-write tensorboard logs
# see https://stackoverflow.com/questions/60079644/how-do-you-edit-an-existing-tensorboard-training-loss-summary
import sys
from pathlib import Path
import os
# Use this if you want to avoid using the GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.core.util.event_pb2 import Event
from tensorflow.errors import DataLossError

In [111]:
trial = trials[1]
for trial in tqdm(trials):
    trial_name = trial.split('/')[-1]
    out_path = f'/home/upload/ray_results/{config_name}_filtered_tb/{trial_name}'
    !rm -rf {out_path}
    os.makedirs(out_path, exist_ok=True)
    tf_files = sorted([x for x in os.listdir(trial) if x.startswith('events.out.tfevents')])
    input_path = os.path.join(trial, tf_files[0])
    assert len(tf_files) == 1, tf_files

    with tf.compat.v1.Graph().as_default():
        writer = tf.compat.v1.summary.FileWriter(out_path)

    tags_ = set()

    try:
        for event in tqdm(tf.compat.v1.train.summary_iterator(input_path), leave=False):
            event_type = event.WhichOneof('what')
            if event_type != 'summary':
                writer.add_event(event)
                pass
            else:
                wall_time = event.wall_time
                step = event.step

        #         possible types: simple_value, image, histo, audio
        #         print(value.tag_hparams)
        #         for value in event.summary.value:
        #             if value.tag.startswith('_hparams'):
        #                 print(value)

                filtered_values = [value for value in event.summary.value if value.HasField('simple_value') or value.tag.startswith('_hparams')]
                summary = tf.compat.v1.Summary(value=filtered_values)

                filtered_event = Event(summary=summary,
                                                  wall_time=wall_time,
                                                  step=step)
        #         print(filtered_event)
                writer.add_event(filtered_event)
    except DataLossError as e:
        print(trial, e)
    writer.flush()
    writer.close()

/home/upload/ray_results/small_tune_ve5_linear_stop_after_completes/main_fcn_48f2d_00007_7__gin__ann_every__choice=100,_gin__batchnorm__choice=False,_gin__fdim__choice=4,_gin__opt__choice=Adam,_gin__2020-11-30_22-09-41 truncated record at 9474036


/home/upload/ray_results/small_tune_ve5_linear_stop_after_completes/main_fcn_92643_00005_5__gin__ann_every__choice=100,_gin__batchnorm__choice=True,_gin__fdim__choice=6,_gin__opt__choice=Adam,_gin___2020-11-28_02-37-12 truncated record at 16975227


In [ ]:
!ls -la {input_path}

In [84]:
tags_

{'_hparams_/experiment',
 '_hparams_/session_end_info',
 '_hparams_/session_start_info',
 'ray/tune/CausalModel',
 'ray/tune/checkpoint_size',
 'ray/tune/epochs',
 'ray/tune/iterations_since_restore',
 'ray/tune/loss_opt1',
 'ray/tune/losses/opt1/fit/coeff',
 'ray/tune/losses/opt1/fit/value',
 'ray/tune/losses/opt1/reconstruction/coeff',
 'ray/tune/losses/opt1/reconstruction/value',
 'ray/tune/losses/opt1/sparsity/coeff',
 'ray/tune/losses/opt1/sparsity/value',
 'ray/tune/losses/opt1/value',
 'ray/tune/metrics/nnz',
 'ray/tune/metrics/sparsity_gap_fit',
 'ray/tune/model',
 'ray/tune/n_samples',
 'ray/tune/perf/cpu_util_percent',
 'ray/tune/perf/ram_util_percent',
 'ray/tune/threshold/action',
 'ray/tune/threshold/feature',
 'ray/tune/threshold_learner',
 'ray/tune/time_since_restore',
 'ray/tune/time_this_iter_s',
 'ray/tune/timesteps_since_restore',
 'ray/tune/weights/decoder/fc.weight',
 'ray/tune/weights/model/fc_action.weight',
 'ray/tune/weights/model/fc_features.weight',
 'ray/tu